# Notebook Grafici
Questo notebook ha lo scopo di creare le funzioni che generano i grafici successivamente utilizzati per la presentazione.

### Cella Import

In [1]:
import matplotlib.pyplot as plt  # https://matplotlib.org/
from matplotlib import ticker  # https://matplotlib.org/stable/api/ticker_api.html
from matplotlib.collections import PolyCollection, PathCollection  # https://matplotlib.org/stable/api/collections_api.html
from matplotlib.widgets import RangeSlider, CheckButtons  # https://matplotlib.org/stable/api/widgets_api.html
from matplotlib.legend_handler import HandlerTuple  # https://matplotlib.org/stable/api/legend_handler_api.html
import colorsys  # https://docs.python.org/3/library/colorsys.html
import math  # https://docs.python.org/3/library/math.html
import numpy as np  # https://numpy.org/
import scipy as sp  # https://scipy.org/
import pandas as pd  # https://pandas.pydata.org/docs/index.html
import seaborn as sns  # https://seaborn.pydata.org/index.html
plt.rcParams['figure.dpi'] = 600  # Aumento i DPI per le visualizzazioni raster

## Fase 1

### Grafico 1

In [2]:
def crea_palette(alpha, hue):
    """
    Metodo utilizzato per creare la palette con i colori per il primo grafico.

    :param alpha: trasparenza del colore
    :param hue: tonalità del colore
    :returns: valore del colore HEX
    """
    
    r, g, b = colorsys.hls_to_rgb(hue, alpha, 1.0)  # Colore RGB
    return '#{:02x}{:02x}{:02x}'.format(int(round(r, 3) * 255), int(round(g, 3) * 255), int(round(b, 3) * 255))  # Colore HEX

palette = pd.Series(range(60, 95, 5)) / 100  # Lista valori alpha
palette = pd.concat([palette.apply(crea_palette, args=(0.0,)),  # 0.0 = Rosso
                     pd.Series(["#ffffff"]),  # Bianco
                     palette.iloc[::-1].apply(crea_palette, args=(1/3,))  # 0.3... = Verde
                    ]).reset_index(drop=True)  # Creazione palette

def colore_delta_start(lista_delta):
    """
    Metodo che data la lista dei valori da rappresentare nelle barre del grafico, ritorna i colori di ciasuna.

    :param lista_delta: lista dei valori delle barre del barplot
    :returns: lista colori delle barre del barplot
    """

    def colore(delta):
        """
        Metodo che dato un singolo valore di una barra, restituisce il suo colore
        
        :param delta: valore di una barra del barplot
        :returns: colore di una barra del barplot
        """
        
        if delta <= - int(palette.size / 2):
            return palette[0]  # Colore estremo negativo
        elif delta >= + int(palette.size / 2):
            return palette[palette.size - 1]  # Colore estremo positivo
        else:
            return palette[delta + int(palette.size / 2)]  # Sfumatura tra colore negativo e positivo
    
    return lista_delta.apply(colore).tolist()

def grafico_partenze_anno_pilota(df_partenze, anno, pilota, dimensioni_grafico):
    """
    Crea il primo grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param anno: anno protagonista del grafico
    :param pilota: id del pilota protagonista del grafico
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """

    df_partenze = df_partenze.loc[(anno, slice(None), pilota)]  # Seleziono Anno, (tutti i round) e Pilota
    esempio_partenza = df_partenze.iloc[0]  # Informazioni Personalizzazione Grafico
    
    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes
    
    sns.barplot(
        ax=ax,
        data=df_partenze,
        x='RoundNumber',
        y='DeltaStart',
        hue='RoundNumber',
        palette=colore_delta_start(df_partenze['DeltaStart']),
        legend=False
    )  # Plotting

    ax.axhline(y=0, color="dimgray", linewidth=2)  # Linea Orizzontale Riferimento
    
    titolo = esempio_partenza['DriverName'] + " - Partenze " + str(anno)
    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure

    ax.set_xlabel(xlabel="N° Gara", fontstyle="italic", labelpad=15)  # Titolo X Axis
    ax.set_ylabel(ylabel="Δ Start", fontstyle="italic", labelpad=15)  # Titolo Y Axis
    
    ymin = min(0, df_partenze['DeltaStart'].min())
    ymax = max(0, df_partenze['DeltaStart'].max())
    ax.set_ylim(ymin - 1/2, ymax + 1/2)  # Limiti Y Axis

    yticks = range(int(ymin), int(ymax) + 1, 1)
    ax.set_yticks(yticks)  # Ticks Y Axis

    if (ymax - ymin) > 12:
        ax.set_yticklabels(str(yt) if yt % 2 == 0 else "" for yt in yticks)  # Tick Labels Y Axis (Spazio Esteso)
    else:
        ax.set_yticklabels(str(yt) for yt in yticks)  # Tick Labels Y Axis (Spazio Ridotto)

    ax.tick_params(axis="x", labelsize=16, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels Y Axis
    ax.tick_params(axis="y", labelsize=16, colors="dimgray", grid_linewidth=0.75)  # Proprietà Labels Y Axis
    
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

### Grafico 1 Esteso

In [3]:
def sub_grafico_partenze_pilota(df_partenze, anno, ax):
    """
    Metodo esegui il plotting dei sotto grafici che compongono la versione estesa del grafico.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1, già filtrato sul pilota
    :param anno: anno protagonista del sotto grafico
    :param ax: asse su cui rappresentare il grafico
    """

    df_partenze = df_partenze.loc[(anno, slice(None))]  # Seleziono Anno e (tutti i round)

    sns.barplot(
        ax=ax,
        data=df_partenze,
        x='RoundNumber',
        y='DeltaStart',
        hue='RoundNumber',
        palette=colore_delta_start(df_partenze['DeltaStart']),
        legend=False
    )  # Plotting

    ax.set_title(label=anno, fontstyle="italic", pad=10)  # Titolo Sub Figure

    for bar in ax.patches:  # Barre Bar Plot
        if abs(bar.get_height()) > 6:  # Valori Fuori Scala
            x = bar.get_x() + (bar.get_width() / 2)  # Coordinata X
            y, colore = (6, "black") if (bar.get_height() > 0) else (-6, "white")  # Coordinata Y e Colore
            testo = str(abs(int(bar.get_height())))  # Testo Text
            dim = 8 if (abs(bar.get_height()) < 10) else 6  # Dimensione (minore se è doppia cifra)
            ax.text(
                x=x,
                y=y,
                s=testo,
                color=colore,
                size=dim,
                ha="center",  # Allineamento Orizzontale
                va="center"  # Allineamento Verticale
            )  # Visualizzazione Testo

def grafico_partenze_pilota(df_partenze, pilota, dimensioni_grafico):
    """
    Crea il primo grafico (versione estesa), della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """

    df_partenze = df_partenze.loc[(slice(None), slice(None), pilota)]  # Seleziono (tutti gli anni), (tutti i round) e Pilota
    esempio_partenza = df_partenze.iloc[0]  # Informazioni Personalizzazione Grafico

    anni = df_partenze.index.get_level_values('Year').unique()  # Elenco Anni Unici
    
    num_colonne = 3  # Colonne SubPlot
    num_righe = math.ceil(len(anni) / num_colonne)  # Righe SubPlot 
    
    fig, ax = plt.subplots(num_righe, num_colonne, sharey=True, figsize=dimensioni_grafico)  # Creazione (Sub)Plot Figure, Axes (Y comune)

    i = 0  # Indice Contatore
    for anno in anni:
        indice_riga = i // num_colonne
        indice_colonna = i % num_colonne

        sub_grafico_partenze_pilota(df_partenze, anno, ax[indice_riga, indice_colonna])  # Sub Plotting
        
        i += 1  # Incremento Contatore

    for i_vuoto in range(i, num_righe * num_colonne):
        fig.delaxes(ax[i_vuoto // num_colonne][i_vuoto % num_colonne])  # Elimino gli eventuali SubPlots Vuoti
        
    titolo = esempio_partenza['DriverName'] + " - Partenze Carriera"
    fig.suptitle(t=titolo, fontweight="bold")  # Titolo SupFigure

    fig.supxlabel(t="N° Gara", fontstyle="italic")  # Titolo Sup X Axis
    fig.supylabel(t="Δ Start", fontstyle="italic")  # Titolo Sup Y Axis

    ymin, ymax = -6, 6  # Limiti Sub Y Axis Globali (sharey)
    yticks = range(ymin, ymax + 1, 1)  # Ticks Sub Y Axis Globali (sharey)
    yticklabels = [str(yt) if yt % 2 == 0 else "" for yt in yticks]  # Tick Labels Sub Y Axis Globali (sharey)
    
    for riga_sub_ax in ax:  # Riga Sub Axes
        for sub_ax in riga_sub_ax:  # Singoli Sub Axes

            sub_ax.axhline(y=0, color="dimgray", linewidth=1)  # Linea Orizzontale Riferimento
            
            sub_ax.set_xlabel(xlabel="")  # Titolo Sub X Axis
            sub_ax.set_ylabel(ylabel="")  # Titolo Sub Y Axis

            sub_ax.set_ylim(ymin - 1/2, ymax + 1/2)  # Limiti Sub Y Axis
            sub_ax.set_yticks(yticks)  # Ticks Sub Y Axis
            sub_ax.set_yticklabels(yticklabels)  # Tick Labels Sub Y Axis
            sub_ax.tick_params(axis="x", labelsize=7, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels Sub Y Axis
            sub_ax.tick_params(axis="y", labelsize=10, colors="dimgray", grid_linewidth=0.5)  # Proprietà Labels Sub Y Axis

    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

### Grafico 2

In [4]:
def filtra_percentile(df, group, colonna, start, end):
    """
    Metodo che permette di filtrare i dati secondi i livelli percentili.

    :param df: Pandas DataFrame generico
    :param group: colonna secondo cui effetturare il raggruppamento (lambda x : True se non si vuole utilizzare questo parametro)
    :param colonna: colonna a cui applicare i filtri dei livelli percentili
    :param start: primo livello percentile sotto il quale escludere i dati
    :param end: secondo livello percentile sopra il quale escludere i dati
    :returns: df a cui è stato applicato il filtro, secondo gruppi e colonna
    """

    lista_df_filtrati = []  # Lista Vuota DF

    for gruppo, df_gruppo in df.groupby(group):
        perc_start, perc_end = np.percentile(df_gruppo[colonna], [start, end])  # Calcolo i livelli percentili
        lista_df_filtrati.append(df_gruppo[(df_gruppo[colonna] >= perc_start) & (df_gruppo[colonna] <= perc_end)])  # Applico il filtro calcolato

    return pd.concat(lista_df_filtrati, axis=0)  # Concateno i DF dei risultati

def get_partenza_compagno(df_partenze, partenza_pilota, pilota):
    """
    Metodo che data una partenza del pilota, estrae quella del suo compagno

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param partenza_pilota: informazioni sulla partenza di un pilota
    :param pilota: id del pilota protagonista del grafico
    :returns: partenza del compagno di squadra (solo se è partito)
    """

    df_partenze = df_partenze.loc[partenza_pilota.name]  # Seleziono Anno e Round
    df_partenze = df_partenze[df_partenze['TeamName'] == partenza_pilota['TeamName']]  # Seleziono Squadra
    partenza = df_partenze.drop(index=pilota)  # Lascio solo il compagno di squadra
    
    return None if partenza.empty else partenza.squeeze()  # Return partenza compagno, oppure None se il compagno non è partito
    

def partenze_pilota_compagni(df_partenze, pilota):
    """
    Metodo che estrae il DataFrame necessario all'analisi del secondo grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :returns: coppia di DataFrame suddivisi tra pilota e compagno di squadra
    """
    
    df_partenze_pilota = df_partenze.loc[(slice(None), slice(None), pilota)]  # Seleziono (tutti gli anni), (tutti i round) e Pilota
    df_partenze_compagno = df_partenze_pilota.copy()  # Il riferimento è quello del pilota

    df_partenze_compagno = df_partenze_compagno.apply(lambda partenza_pilota : get_partenza_compagno(df_partenze, partenza_pilota, pilota), 
                                                      axis=1)  # Si cerca la partenza del compagno a partire dal pilota
    df_partenze_compagno = df_partenze_compagno.dropna(how="all")  # Significa che il compagno di squadra non ha partecipato alla gara

    return df_partenze_pilota, df_partenze_compagno  # Return coppia di dataframe pilota-compagno

def tabella_partenze_pilota_compagni(df_partenze, pilota):
    """
    Crea la tabella associata al secondo grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :returns: tabella con informazioni su pilota e compagni
    """
    
    df_partenze_pilota, df_partenze_compagni = partenze_pilota_compagni(df_partenze, pilota)  # Seleziono partenze pilota e compagni

    esempio_partenza = df_partenze_pilota.iloc[0]  # Informazioni Personalizzazione Grafico

    # Nomi Colonne
    compagno = 'Compagno Squadra'
    punti_pilota = 'Punti ' + esempio_partenza['DriverAbbreviation']
    punti_compagno = 'Punti Compagno'
    perc_pilota = '% ' + esempio_partenza['DriverAbbreviation']
    perc_compagno = '% Comp.'
    griglia_pilota = 'Griglia μ ' + esempio_partenza['DriverAbbreviation']
    griglia_compagno = 'Griglia μ Compagno'
    delta_pilota = 'Δ Start μ ' + esempio_partenza['DriverAbbreviation']
    delta_compagno = 'Δ Start μ Compagno'

    df_tabella_sx = pd.DataFrame(data=df_partenze_pilota,
                                 columns=['Points', 'GridPosition', 'DeltaStart'])  # Seleziono le colonne d'interesse

    df_tabella_sx.rename(columns={'Points': punti_pilota,
                                  'GridPosition': griglia_pilota,
                                  'DeltaStart': delta_pilota},
                         inplace=True)  # Rinomino le colonne
    
    df_tabella_sx = df_tabella_sx.groupby(level='Year').agg({punti_pilota: "sum",
                                                             griglia_pilota: "mean",
                                                             delta_pilota: "mean"})  # Applico le funzioni di aggregazione

    df_tabella_dx = pd.DataFrame(data=df_partenze_compagni,
                                 columns=['DriverName', 'Points', 'GridPosition', 'DeltaStart'])  # Seleziono le colonne d'interesse

    df_tabella_dx.rename(columns={'DriverName': compagno,
                                  'Points': punti_compagno,
                                  'GridPosition': griglia_compagno,
                                  'DeltaStart': delta_compagno},
                         inplace=True)  # Rinomino le colonne

    df_tabella_dx = df_tabella_dx.groupby(by=['Year', compagno]).agg({punti_compagno: "sum",
                                                                      griglia_compagno: "mean",
                                                                      delta_compagno: "mean"})  # Applico le funzioni di aggregazione
    
    df_tabella_dx = df_tabella_dx.reset_index(drop=False).set_index('Year')  # Un pilota potrebbe avere diversi compagni di squadra in un anno

    df_tabella_dx = df_tabella_dx.groupby('Year').agg({compagno: lambda x: "; ".join(x),
                                                       punti_compagno: "sum",
                                                       griglia_compagno: "mean",
                                                       delta_compagno: "mean"})  # Riapplico le funzioni di aggregazione, unendo i compagni di squadra

    
    df_tabella = pd.merge(df_tabella_sx, df_tabella_dx, on='Year')  # Eseguo un merge (inner join, gli anni ovviamente coincidono)

    df_tabella['Tot'] = df_tabella[punti_pilota] + df_tabella[punti_compagno]  # Calcolo il totale dei punti
    df_tabella[perc_pilota] = df_tabella[punti_pilota] / df_tabella['Tot'] * 100  # Calcolo % punti pilota 
    df_tabella[perc_compagno] = df_tabella[punti_compagno] / df_tabella['Tot'] * 100  # Calcolo % punti compagno
    
    df_tabella[punti_compagno] = df_tabella[punti_compagno].round(1)  # Arrotondamento Punti Compagno
    df_tabella[perc_compagno] = df_tabella[perc_compagno].round(1)  # Arrotondamento % Punti Compagno
    df_tabella[perc_pilota] = df_tabella[perc_pilota].round(1)  # Arrotondamento % Punti Pilota
    df_tabella[punti_pilota] = df_tabella[punti_pilota].round(1)  # Arrotondamento Punti Pilota
    
    df_tabella[griglia_pilota] = df_tabella[griglia_pilota].round(1)  # Arrotondamento Media Griglia Pilota
    df_tabella[delta_pilota] = df_tabella[delta_pilota].round(2)  # Arrotondamento Media DeltaStart Pilota
    df_tabella[delta_compagno] = df_tabella[delta_compagno].round(2)  # Arrotondamento Media DeltaStart Compagno
    df_tabella[griglia_compagno] = df_tabella[griglia_compagno].round(1)  # Arrotondamento Media Griglia Compagno
    
    df_tabella = df_tabella.loc[:, [compagno,
                                    punti_compagno, perc_compagno, perc_pilota, punti_pilota,  # Seleziono nuovamente le colonne (eliminando il totale)
                                    griglia_pilota, delta_pilota, delta_compagno, griglia_compagno]]  # dando l'ordine desiderato per l'output

    df_tabella.index.names = ['Anno']  # Rinomino l'indice
    
    return df_tabella  # Return tabella

def grafico_partenze_pilota_compagni(df_partenze, pilota, perc_start, perc_end, dimensioni_grafico):
    """
    Crea il secondo grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :param perc_start: livello percentile minimo da applicare eventualmente alle partenze
    :param perc_end: livello percentile massimo da applicare eventualmente alle partenze
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """

    df_partenze_pilota, df_partenze_compagni = partenze_pilota_compagni(df_partenze, pilota)  # Seleziono partenze pilota e compagni
    
    esempio_partenza = df_partenze_pilota.iloc[0]  # Informazioni Personalizzazione Grafico
    df_partenze_compagni['DriverName'] = "Compagno"  # Uniformo il compagno per la Hue

    df_partenze_pilota = filtra_percentile(df_partenze_pilota, 'Year', 'DeltaStart', perc_start, perc_end)  # Applico il filtro percentile
    df_partenze_compagni = filtra_percentile(df_partenze_compagni, 'Year', 'DeltaStart', perc_start, perc_end)  # Applico il filtro percentile

    df_partenze = pd.concat([df_partenze_pilota, df_partenze_compagni], axis=0)  # Combino i DataFrame
    
    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes

    sns.violinplot(
        ax=ax,
        data=df_partenze,
        x='Year',
        y='DeltaStart',
        hue='DriverName',
        palette=['0.25', '0.75'],
        split=True,
        gap=0.05,
        fill=True,
        width=0.9,
        bw_adjust=0.9,
        cut=2,
        density_norm="count",
        inner="quart",  # aggiungo i quartili
        inner_kws={"color": "black"}
    )  # Plotting

    serie_etichette = df_partenze_compagni.groupby('Year')['DriverAbbreviation'].apply(lambda x: x.value_counts().idxmax())  # Selezione Etichette
    dizionario_mapping = {etichetta: indice for indice, etichetta in enumerate(serie_etichette.unique())}  # Dizionario Mapping Etichette -> Colori
    serie_colori = serie_etichette.map(dizionario_mapping)  # Selezione Colori
    df_etichette_colori = pd.DataFrame(data={'Etichette': serie_etichette, 'Colori': serie_colori})  # Si uniscono colori e etichette in un DF
    palette = sns.color_palette("Set2")  # Palette Hue

    identificatori_legenda = []  # Lista degli identificatori che comporranno la legenda
    for indice, violino in enumerate(ax.findobj(PolyCollection)):
        colore = palette[df_etichette_colori['Colori'].iloc[indice // 2]]  # Si seleziona il colore nella palette
        if indice % 2 != 0:
            colore = 0.5 + 0.5 * np.array(colore)  # Si schiarisce il colore in base alla Hue
        violino.set_facecolor(colore)  # Si imposta il colore al violino
        identificatori_legenda.append(plt.Rectangle((0, 0), 0, 0, facecolor=colore, edgecolor="black"))  # Creo l'identificatore del violino
    
    titolo = esempio_partenza['DriverName'].split(" ")[1] + " vs. Compagni - Partenze"
    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure
    
    ax.set_xlabel(xlabel="Anno - Compagno", fontstyle="italic", labelpad=15)  # Titolo X Axis
    ax.set_ylabel(ylabel="Δ Start", fontstyle="italic", labelpad=15)  # Titolo Y Axis

    ymin = math.ceil(ax.get_ylim()[0])
    ymax = math.floor(ax.get_ylim()[1])

    ax.set_xticks(ax.get_xticks())  # Ticks X Axis
    yticks = range(int(ymin), int(ymax) + 1, 1)
    ax.set_yticks(yticks)  # Ticks Y Axis

    xticklabels = ax.get_xticklabels()
    
    for xt in xticklabels:
        anno = int(xt.get_text())
        xt.set_text(xt.get_text() + " - " + df_etichette_colori.loc[anno, 'Etichette'])  

    ax.set_xticklabels(xticklabels)  # Tick Labels X Axis
    
    if (ymax - ymin) > 12:
        ax.set_yticklabels(str(yt) if yt % 2 == 0 else "" for yt in yticks)  # Tick Labels Y Axis (Spazio Esteso)
    else:
        ax.set_yticklabels(str(yt) for yt in yticks)  # Tick Labels Y Axis (Spazio Ridotto)

    ax.tick_params(axis="x", labelsize=12, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels Y Axis
    ax.tick_params(axis="y", labelsize=16, colors="dimgray", grid_linewidth=0.75)  # Proprietà Labels Y Axis

    legenda_labels = df_partenze['DriverName'].unique().tolist()  # Elenco legenda labels
    legenda_labels = [pilota.split()[-1] if len(pilota.split()) > 1 else pilota for pilota in legenda_labels]  # Riduco le labels a 1 lettera
    legenda = ax.legend(
        title="Pilota",
        handles=[tuple(identificatori_legenda[::2]), tuple(identificatori_legenda[1::2])],
        handlelength=(len(identificatori_legenda) // 2),
        handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
        labels=legenda_labels,
        fontsize=10
    )  # Proprietà Legenda
    for testo in legenda.texts:
        testo.set_fontsize(10)  # Proprietà Label Legenda
        
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

### Grafico 3

In [5]:
def tabella_andamento_partenze_pilota(df_partenze, pilota):
    """
    Crea la tabella associata al terzo grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :returns: tabella con informazioni sul pilota nel corso degli anni
    """

    df_partenze = df_partenze.loc[(slice(None), slice(None), pilota)]  # Seleziono (tutti gli anni), (tutti i round) e Pilota
    
    df_tabella = pd.DataFrame(data=df_partenze, columns=['GridPosition', 'DeltaStart', 'ClassifiedPosition'])  # Seleziono le colonne d'interesse
    
    df_tabella = df_tabella.rename(
        columns={'GridPosition': 'P. Partenza μ', 'DeltaStart': 'Δ Start μ', 'ClassifiedPosition': 'P. Arrivo μ'})  # Rinomino le colonne
        
    df_tabella = df_tabella.groupby('Year').mean()  # Applico le funzioni di aggregazione

    df_tabella.index.name = 'Anno'  # Rinomino l'index

    df_tabella = df_tabella.round(2)  # Arrotondo la tabella alla seconda cifra decimale
    
    return df_tabella  # Return tabella

def grafico_andamento_partenze_pilota(df_partenze, pilota, dimensioni_grafico):
    """
    Crea il terzo grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """

    df_partenze = df_partenze.loc[(slice(None), slice(None), pilota)]  # Seleziono (tutti gli anni), (tutti i round) e Pilota
    esempio_partenza = df_partenze.iloc[0]  # Informazioni Personalizzazione Grafico

    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes

    sns.boxenplot(
        ax=ax,
        data=df_partenze,
        x='Year',
        y='DeltaStart',
        hue='Year',
        palette=sns.color_palette("Spectral", n_colors=len(df_partenze.index.unique(level='Year'))),
        width_method="linear",
        k_depth=4,
        line_kws=dict(linewidth=2.0, color="black", linestyle="dashed"),
        showfliers=True,
        flier_kws=dict(facecolor="dimgray", linewidth=0.5),
        legend=False
    )  # Plotting

    titolo = esempio_partenza['DriverName'] + " - Andamento Partenze"
    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure

    ax.set_xlabel(xlabel="Anno", fontstyle="italic", labelpad=15)  # Titolo X Axis
    ax.set_ylabel(ylabel="Δ Start", fontstyle="italic", labelpad=15)  # Titolo Y Axis

    ymin = min(0, df_partenze['DeltaStart'].min())
    ymax = max(0, df_partenze['DeltaStart'].max())
    yticks = range(int(ymin) - 1, int(ymax) + 2, 1)
    
    ax.set_yticks(yticks)  # Ticks Y Axis
    ax.set_yticklabels(str(yt) if yt % 5 == 0 else "" for yt in yticks)  # Tick Labels Y Axis (Spazio Esteso)
    
    ax.tick_params(axis="x", labelsize=16, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels X Axis
    ax.tick_params(axis="y", labelsize=16, colors="dimgray", grid_linewidth=0.5)  # Proprietà Labels Y Axis
    
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

### Grafico 4

In [6]:
def grafico_distribuzione_partenze_pilota(df_partenze, pilota, pos_min, pos_max, perc_start, perc_end, dimensioni_grafico):
    """
    Crea il quarto grafico, della prima parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param pilota: id del pilota protagonista del grafico
    :param pos_min: posizione di partenza minima del pilota
    :param pos_max: posizione di partenza massima del pilota
    :param perc_start: livello percentile minimo da applicare eventualmente alle partenze
    :param perc_end: livello percentile massimo da applicare eventualmente alle partenze
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """
    
    df_partenze = df_partenze.loc[(slice(None), slice(None), pilota)]  # Seleziono (tutti gli anni), (tutti i round) e Pilota
    esempio_partenza = df_partenze.iloc[0]  # Informazioni Personalizzazione Grafico

    df_partenze = df_partenze[(df_partenze['GridPosition'] >= pos_min) & (df_partenze['GridPosition'] <= pos_max)]  # Applico il filtro di posizione
    df_partenze = filtra_percentile(df_partenze, 'GridPosition', 'Lap1Position', perc_start, perc_end)  # Applico il filtro percentile

    n_colori = pos_max - pos_min + 1  # Numero dei colori della palette
    palette = sns.color_palette("PuBuGn_r", n_colori * 2)[:n_colori:]  # Palette Colori
    
    g = sns.FacetGrid(
        data=df_partenze,
        row='GridPosition',
        hue='GridPosition',
        aspect=dimensioni_grafico[0],
        height=dimensioni_grafico[1],
        sharex=True,
        sharey=False,
        palette=palette
    )  # Creazione Grafico Seaborn 
    fig = g.figure  # Oggetto Plot Figure
    ax = g.axes  # Oggetto Plot Axes

    def grafico_riferimenti_annotazioni(posizioni_griglia, delta_start, color, label):
        """
        Metodo che crea le annotazioni dei sotto grafici.

        :param posizioni_griglia: colonna GridPosition associata al raggruppamento per GridPosition
        :param delta_start: colonna DeltaStart associata al raggruppamento per GridPosition
        :param color: colore del valore sull'asse Y
        :param label: etichetta del valore sull'asse Y
        """

        ax = fig.gca()  # Axis Attivo

        posizione = int(float(label))  # Posizione (Valore Sup Y)
        occorrenze_posizione = posizioni_griglia.size  # Contatore numero occorrenze
        media_delta_start = round(delta_start.mean(), 2)  # Media posizione perse/guadagnate al via

        ax.text(
            x=0,
            y=0.5,
            s=" " + str(posizione),
            size=16,
            fontweight="bold",
            color=color,
            ha="left",
            va="center",
            transform=ax.transAxes
        )  # Posizione (Valore Sup Y)

        ax.axvline(
            x=posizione,
            c="black",
            ls="--",
            linewidth=2.0
        )  # Riferimento rispetto alla posizione di partenza

        ax.text(
            x=0.075,
            y=0.5,
            s="×" + str(occorrenze_posizione),
            size=10,
            fontweight="bold",
            color="black",
            ha="left",
            va="center",
            transform=ax.transAxes
        )  # Contatore numero occorrenze

        ax.text(
            x=1,
            y=0.5,
            s="{:.2f}".format(media_delta_start) + " ",
            size=16,
            fontweight="bold",
            color=color,
            ha="right",
            va="center",
            transform=ax.transAxes
        )  # Media posizione perse/guadagnate al via
    
    g.map(
        sns.kdeplot,
        'Lap1Position',
        bw_adjust=0.75,
        fill=True,
        alpha=1.0,
        clip_on=False
    )  # Plotting KDE Plot interno filled
    g.map(
        sns.kdeplot,
        'Lap1Position',
        bw_adjust=0.75,
        color="white",
        linewidth=3.0,
        clip_on=False
    )  # Plotting KDE Plot linea contorno bianca
    g.refline(
        y=0.0,
        linewidth=1.5,
        linestyle="-",
        color=None,
        clip_on=False)  # Plotting Linea Riferimento y=0.0
    g.map(
        grafico_riferimenti_annotazioni,
        'GridPosition',
        'DeltaStart'
    )  # Plotting Riferimenti - Annotazioni (Posizioni, Occorrenze, DeltaStart e Linee Verticali)
    
    fig.text(0.13, 0.92, "Occorrenze", fontsize=10, fontstyle="italic")  # Testo in alto a sinistra (occorrenze)
    fig.text(0.90, 0.92, "Δ Start μ", fontsize=10, fontstyle="italic")  # Testo in alto a destra (DeltaStart)
    
    g.set_titles("")  # Rimozione dei titoli dei singoli grafici
    g.despine(bottom=True, left=True)  # Rimozione delle spine dal grafico
    
    titolo = esempio_partenza['DriverName'] + " - Distribuzione Partenze"
    fig.suptitle(t=titolo, fontsize=20, fontweight="bold")  # Titolo SupFigure

    fig.supxlabel(t="P. Giro 1", fontsize=16, fontstyle="italic")  # Titolo Sup X Axis
    fig.supylabel(t="P. Partenza", fontsize=16, fontstyle="italic")  # Titolo Sup Y Axis

    xmin = min(pos_min, max(1, pos_min - 5))
    xmax = max(pos_max, min(20, pos_max + 5))
    xticks = range(xmin, xmax + 1, 1)
    g.set(xticks=xticks, xlabel="")  # Ticks e Labels X Axis
    g.set(yticks=[], ylabel="")  # Ticks e Labels Y Axis

    g.tick_params(axis="x", labelsize=16, colors="dimgray", grid_linewidth=1.0)  # Proprietà Labels X Axis
    g.tick_params(axis="y", labelsize=16, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels Y Axis
    
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

## Fase 2

### Grafico 1

In [7]:
def grafico_heatmap_posizioni(df_partenze, titolo, righe, nome_righe, colonne, nome_colonne, mappa_colori, dimensioni_grafico):
    """
    Crea il secondo grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param titolo: titolo del grafico
    :param righe: colonna del DF da rappresentare "orizzontalmente" nel grafico
    :param nome_righe: nome associato alle righe da visualizzare sull'asse Y
    :param colonne: colonna del DF da rappresentare "verticalmente" nel grafico
    :param nome_colonne: nome associato alle colonne da visualizzare sull'asse X
    :param mappa_colori: nome della CMAP da utilizzare per il grafico
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """
    
    df_partenze = df_partenze.groupby([righe, colonne]).size()  # Raggruppo per righe-colonne e conto le occorrenze
    df_partenze = df_partenze.reset_index(drop=False, name='Count')  # Rimuovo l'indice righe-colonne e nomino le occorrenze

    df_partenze = df_partenze.pivot(index=righe, columns=colonne, values='Count')  # Eseguo il pivoting della tabella

    df_partenze = df_partenze.fillna(0)  # Riempio i valori nan con 0, cioè nessuna occorrenza di questa posizione
    
    totali_riga = df_partenze.sum(axis=1)  # Serie che in ciascun indice ha il totale della rispettiva riga della tabella
    
    for df_colonna in df_partenze.columns:
        df_partenze[df_colonna] = df_partenze[df_colonna] / totali_riga  # Calcolo la % rispetto alla riga

    maschera_df = df_partenze < 0.001  # Maschera da applicare al dataframe: nasconde i valori inferiori allo 0.1%
    
    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes

    sns.heatmap(
        ax=ax,
        data=df_partenze,
        mask=maschera_df,
        yticklabels=1,
        linewidth=0.5,
        annot=True,
        annot_kws={"fontsize": 8, "multialignment": "center"},
        fmt=".1%",
        cmap=sns.color_palette(mappa_colori, as_cmap=True),
        cbar=True,
        cbar_kws={"shrink": 0.75}
    )  # Plotting

    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure

    ax.set_xlabel(xlabel=nome_colonne, fontstyle="italic", labelpad=15)  # Titolo X Axis
    ax.set_ylabel(ylabel=nome_righe, fontstyle="italic", labelpad=15)  # Titolo Y Axis

    ax.tick_params(axis="x", labelsize=10, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels X Axis
    ax.tick_params(axis="y", labelsize=10, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels Y Axis

    colorbar = ax.collections[0].colorbar  # Colorbar Cmap %
    colorbar.formatter = ticker.FuncFormatter(lambda x, pos : "{}%".format(int(x * 100)))  # Funzione Formatter Tick
    colorbar.ax.tick_params(axis="y", labelsize=10)  # Tick Labels Colorbar
    colorbar.update_ticks()  # Aggiornamento Ticks
    
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

### Grafico 2

In [8]:
def df_scenario_gomme(df_partenze):
    """
    Metodo che estrae il DataFrame necessario all'analisi del secondo grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :returns: coppia di DataFrame con i due scenari dell'analisi
    """
    
    df_partenze = df_partenze.loc[(slice(2019, None), slice(None), slice(None))]  # I dati sulle gomme sono presenti interamente solo dalla stagione '19
    
    codice_gomme = {
        "SOFT": 3,
        "MEDIUM": 2,
        "HARD": 1
    }  # A ogni compound gomma associo un codice
    df_partenze['Compound'] = df_partenze['Compound'].map(codice_gomme)  # Sostituisco il compound con il suo codice

    df_partenze = df_partenze.reset_index(level='DriverId')  # Riporto l'indice pilota nei dati, così posso iterare su anno e numero gara

    lista_partenze = []  # Lista vuota DF
    for (anno, gara) in df_partenze.index.unique():
        df_partenze_anno_gara = df_partenze.loc[(anno, gara)]  # Itero sul multi indice anno-gara

        df_partenze_anno_gara = df_partenze_anno_gara.sort_values(by='GridPosition')  # Riordino i valori della colonna in base alla posizione in griglia

        df_partenze_anno_gara_shifted = df_partenze_anno_gara.shift(1)  # Per confrontare un pilota con colui che lo precede, shifto di 1 il DF

        df_partenze_anno_gara['CompoundAhead'] = df_partenze_anno_gara_shifted['Compound']  # Salvo il valore del pilota davanti
        df_partenze_anno_gara['Lap1PositionAhead'] = df_partenze_anno_gara_shifted['Lap1Position']  # Salvo il valore del pilota davanti
        df_partenze_anno_gara['FreshTyreAhead'] = df_partenze_anno_gara_shifted['FreshTyre']  # Salvo il valore del pilota davanti
        
        lista_partenze.append(df_partenze_anno_gara.iloc[1:])  # Aggiungo il DF alla lista, senza il 1° (privo di senso per questa analisi)

    df_partenze = pd.concat(lista_partenze, axis=0)  # Unisce i DF
        
    df_partenze_softer = df_partenze[df_partenze['Compound'] > df_partenze['CompoundAhead']].copy()  # Filtro le partenza con gomma più morbida
    df_partenze_fresher= df_partenze[df_partenze['FreshTyre'] > df_partenze['FreshTyreAhead']].copy()  # Filtro le partenza con gomma più fresca

    df_partenze_softer['DeltaAhead'] = df_partenze_softer['Lap1PositionAhead'] - df_partenze_softer['Lap1Position']  # Calcolo la differenza con il pilota
    df_partenze_fresher['DeltaAhead'] = df_partenze_fresher['Lap1PositionAhead'] - df_partenze_fresher['Lap1Position']  # che partiva avanti (entrambi DF)

    return df_partenze_softer, df_partenze_fresher  # Return dei due dataframe

def tabella_scenario_gomme(df_partenze):
    """
    Crea la tabella legata al secondo grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :returns: tabella con informazioni sui due scenari
    """
    
    df_partenze_softer, df_partenze_fresher = df_scenario_gomme(df_partenze.copy())  # Applico la funzione che estrai i due sotto DF

    df_partenze_softer['Scenario'] = "Gomme Più Morbide"  # Assegno il nome allo scenario
    df_partenze_fresher['Scenario'] = "Gomme Più Fresche"  # Assegno il nome allo scenario

    df_tabella = pd.concat([df_partenze_softer, df_partenze_fresher], axis = 0)  # Unisco i due DataFrame

    df_tabella = df_tabella.groupby('Scenario').agg({'Scenario': "count",
                                                     'DeltaAhead': "mean",
                                                     'DeltaStart': "mean"})  # Applico le funzioni di aggregazione per creare la tabella

    df_tabella = round(df_tabella, 2)  # Arrotondo la tabella (medie) alla seconda cifra decimale
    
    df_tabella = df_tabella.rename(columns={'Scenario': 'Occorrenze',
                                            'DeltaAhead': 'Δ Pilota Davanti μ', 'DeltaStart': 'Δ Start μ'})  # Rinomino le colonne
    
    df_tabella.index.name = 'Scenario'  # Imposto il nome dell'indice
    
    return df_tabella  # Return tabella
    
def grafico_scenario_gomme(df_partenze, dimensioni_grafico):
    """
    Crea il secondo grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """

    df_partenze_softer, df_partenze_fresher = df_scenario_gomme(df_partenze.copy())  # Applico la funzione che estrai i due sotto DF
    
    df_partenze_softer = filtra_percentile(df_partenze_softer, lambda x : True, 'DeltaAhead', 10, 90)  # Applico i filtri percentile
    df_partenze_fresher = filtra_percentile(df_partenze_fresher, lambda x : True, 'DeltaAhead', 10, 90)  # Applico i filtri percentile
    
    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes

    colori = sns.color_palette("coolwarm", 8)  # Suddivido la palette, della quale prenderò i due valori estremi
    
    sns.kdeplot(
        ax=ax,
        data=df_partenze_softer,
        x='DeltaAhead',
        bw_adjust=0.8,
        color=colori[-1],
        linewidth=2,
        alpha=0.5,
        fill=True,
        label="Gomme Più Morbide",
        zorder=2
    )  # Plotting Primo KDE
    
    sns.kdeplot(
        ax=ax,
        data=df_partenze_fresher,
        x='DeltaAhead',
        bw_adjust=0.8,
        color=colori[0],
        linewidth=2,
        alpha=0.5,
        fill=True,
        label="Gomme Più Fresche",
        zorder=3
    )  # Plotting Secondo KDE
    
    ax.axvline(x=0, color="dimgray", linestyle="-", linewidth=0.5, zorder=1)  # Linea di riferimento verticale

    titolo = "Scenario Gomme"
    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure

    ax.set_xlabel(xlabel="Δ Pilota Davanti", fontstyle="italic", labelpad=15)  # Titolo X Axis
    ax.set_ylabel(ylabel="Densità", fontstyle="italic", labelpad=15)  # Titolo Y Axis
    
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: "{:.1f}%".format(y * 100)))  # Funzione Formatter Tick, Asse Y
    
    ax.tick_params(axis="x", labelsize=16, colors="dimgray", grid_linewidth=1.0)  # Proprietà Labels X Axis
    ax.tick_params(axis="y", labelsize=16, colors="dimgray", grid_linewidth=1.0)  # Proprietà Labels Y Axis 

    legenda = ax.legend(title="KDE", loc="upper right", fontsize=10)  # Titolo Legenda
    for testo in legenda.texts:
        testo.set_fontsize(10)  # Proprietà Label Legenda
    
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax

### Grafico 3

In [9]:
def grafico_classifica_partenze(df_partenze, dimensioni_grafico):
    """
    Crea il terzo grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """

    df_partenze = df_partenze.groupby('DriverId').agg({'DriverAbbreviation': "last",
                                                       'DriverName': "last",
                                                       'Style': "last",
                                                       'GridPosition': "mean",
                                                       'DeltaStart': "mean",
                                                       'ClassifiedPosition': "count"
                                                       })  # Dopo aver raggrupato per piloti, applico le aggregazioni per ottenere la classifica

    lista_categorie = ["≤1 stagione", "≤3 stagioni", "4+ stagioni"]  # Definisco una lista di categorie per il count

    def assegna_categoria(numero_partenze):
        """
        Metodo che assegna la categoria al numero di partenze di un pilota.
        
        :param numero_partenze: numero di partenze del singolo pilota
        :returns: categoria a cui appartiene il singolo numero di partenze
        """
        
        if numero_partenze <= 25:  # Una stagione, circa 20 gare
            return lista_categorie[0]
        elif numero_partenze <= 80:  # Quattro stagioni, circa 80 gare
            return lista_categorie[1]
        else:  # Più di quattro stagioni
            return lista_categorie[2]

    df_partenze['CategoriaPartenze'] = df_partenze['ClassifiedPosition'].apply(assegna_categoria)  # Applico la funzione di categorizzazione

    serie_annotazioni = pd.Series(data=[])  # Creo una serie vuota dove memorizzare le annotazioni

    def plotting_scatter(df_partenze):
        """
        Metodo che crea il grafico con il plotting di marker, etichette e annotazioni
        
        :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1, filtrato sui valori di GridPosition
        """

        for annotazione in ax.texts:
            annotazione.remove()  # Rimuovo i testi sopra i marker
        for marker in ax.collections:
            marker.remove()  # Rimuovo i marker sul grafico

        df_partenze = df_partenze[df_partenze['CategoriaPartenze'].isin(check_buttons_categoria.get_checked_labels())]
        # Filtro il DataFrame secondo le categorie abilitate

        delta_min, delta_max = df_partenze['DeltaStart'].min(), df_partenze['DeltaStart'].max()  # Calcolo i valori limite dell'asse Y
        y_padding = (delta_max - delta_min) * 0.05  # Calcolo il padding dell'asse Y
        ax.set_ylim(delta_min - y_padding, delta_max + y_padding)  # Applico i nuovi limiti

        for indice, pilota in df_partenze.iterrows():

            ax.scatter(
                x=pilota['GridPosition'],
                y=pilota['DeltaStart'],
                c=pilota['Style']['color'],
                alpha=0.8,
                marker=pilota['Style']['marker'],
                facecolor=pilota['Style']['edgecolor'],
                s=200 * math.pow(delta_max - delta_min, -1),
                gid=indice
            )  # Plotting Piloti

            ax.text(
                x=pilota['GridPosition'],
                y=pilota['DeltaStart'] + math.pow(delta_max - delta_min, -0.5) * 0.1,
                s=pilota['DriverAbbreviation'],
                fontsize=14 * math.pow(delta_max - delta_min, -0.25),
                fontweight="bold",
                ha="center",
                c="black",
                transform=ax.transData
            )  # Plotting Identificatori

            annotazione = ""   # Creo il testo dell'annotazione
            annotazione += pilota['DriverName'] + "\n"
            annotazione += "P. Partenza μ = " + str(round(pilota['GridPosition'], 2)) + "\n"
            annotazione += "Δ Start μ = " + str(round(pilota['DeltaStart'], 2)) + "\n"
            annotazione += "N° Partenze = " + str(pilota['ClassifiedPosition'])

            serie_annotazioni.loc[indice] = ax.annotate(
                text=annotazione,
                xy=(pilota['GridPosition'], pilota['DeltaStart']),
                xytext=(-140, 0),
                textcoords="offset points",
                visible=False,
                zorder=10,
                fontsize=12,
                bbox=dict(boxstyle="round", facecolor="white"),
                arrowprops=dict(arrowstyle="<-", edgecolor="black")
            )  # Plotting Annotazioni Esplicative

        fig.canvas.draw_idle()  # Come ultima cosa, aggiorno la figura

    def aggiorna_check_buttons(val):
        """
        Metodo invocato ogni qual volta viene eseguita un'azione sui check buttons
        
        :param val: valore dei check buttons
        """

        grid_min, grid_max = ax.get_xlim()[0], ax.get_xlim()[1]  # Ricavo i limiti Asse X
        plotting_scatter(df_partenze[(df_partenze['GridPosition'] >= grid_min) &
                                     (df_partenze['GridPosition'] <= grid_max)].copy())  # Ricreo il plotting con i nuovi limiti

    def aggiorna_slider(val):
        """
        Metodo invocato ogni qual volta viene eseguita un'azione sul range slider
        
        :param val: valore dello slider
        """

        grid_min, grid_max = slider_posizione_partenza.val  # Ricavo i limiti Asse X
        ax.set_xlim(grid_min, grid_max)  # Applico i nuovi limiti
        plotting_scatter(df_partenze[(df_partenze['GridPosition'] >= grid_min) &
                                     (df_partenze['GridPosition'] <= grid_max)].copy())  # Ricreo il plotting con i nuovi limiti

    ultima_annotazione = None  # Variabile sentinella che punta all'ultima annotazione

    def gestisci_hover(event):
        """
        Metodo invocato ogni qual volta si passa il mouse all'interno della figura
        
        :param event: evento hover
        """

        nonlocal ultima_annotazione
        if ultima_annotazione is not None:
            serie_annotazioni.loc[ultima_annotazione].set_visible(False)  # Nascondo l'annotazione
            ultima_annotazione = None  # Reimposto il riferimento all'annotazione
        for child in ax.get_children():
            if isinstance(child, PathCollection):
                if child.contains(event)[0]:
                    pilota = child.get_gid()  # Ottengo l'id pilota, collegato all'annotazione
                    serie_annotazioni.loc[pilota].set_visible(True)  # Visualizzo l'annotazione
                    ultima_annotazione = pilota  # Salvo il riferimento all'annotazione

        fig.canvas.draw_idle()  # Come ultima cosa, aggiorno la figura

    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes

    fig.subplots_adjust(left=0.15)  # Aggiungo spazio a sinistra della figura
    fig.subplots_adjust(bottom=0.20)  # Aggiungo spazio al di sotto della figura

    check_buttons_ax = fig.add_axes((0.0, 0.0, 0.152, 0.202))  # Creo lo spazio per il gruppo di check buttons
    check_buttons_categoria = CheckButtons(ax=check_buttons_ax, labels=lista_categorie, actives=[True] * len(lista_categorie))  # Creo i check buttons
    check_buttons_categoria.on_clicked(aggiorna_check_buttons)  # Imposto l'azione da eseguire al click dei buttons
    for etichetta in check_buttons_categoria.labels:
        etichetta.set_fontsize(12)  # Imposto la dimensione del testo delle etichette
        etichetta.set_fontstyle("italic")  # Imposto lo stile del testo delle etichette
    fig.text(x=0.075, y=0.21, s="Filtro N° Gare", fontstyle="italic", fontsize=12, ha="center", transform=fig.transFigure)  # Titolo del gruppo

    x_min, x_max = math.floor(df_partenze['GridPosition'].min()), math.ceil(df_partenze['GridPosition'].max())
    slider_ax = fig.add_axes((0.2125, 0.04, 0.6, 0.035))  # Creo lo spazio per lo slider
    slider_ax.set_title("P. Partenza μ", fontsize=16, fontstyle='italic')  # Imposto il titolo all'asse/slider
    slider_posizione_partenza = RangeSlider(slider_ax, "", x_min, x_max, valinit=(x_min, x_max), color="dimgray", track_color="lightgray")
    # Creo lo slider
    slider_posizione_partenza.on_changed(aggiorna_slider)  # Imposto l'azione da eseguire al cambio del range

    fig.canvas.mpl_connect('motion_notify_event', gestisci_hover)  # Imposto una funzione associata all'hover nel grafico

    plotting_scatter(df_partenze.copy())  # Primo Plotting

    coefficienti_regressione = np.polyfit(df_partenze['GridPosition'], df_partenze['DeltaStart'], 1)  # Calcolo i coeff. polinomiali, funzione 1° grado
    funzione_regressione = np.poly1d(coefficienti_regressione)  # Ricava la funzione polinomiale per la regressione, a partire dai coefficienti
    spazio_lineare = np.linspace(df_partenze['GridPosition'].min(), df_partenze['GridPosition'].max(), 1000)  # Creo lo spazio lineare per i "valori x"
    ax.plot(spazio_lineare, funzione_regressione(spazio_lineare), linewidth=1.0, color="dimgray")  # Rappresento la funzione di regressione

    titolo = "Classifica Partenze Piloti"
    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure

    # ax.set_xlabel(xlabel="P. Partenza μ", fontstyle="italic", labelpad=15)  # Titolo X Axis -> Rimosso, incluso nello slider
    ax.set_ylabel(ylabel="Δ Start μ", fontstyle="italic", labelpad=15)  # Titolo Y Axis

    ax.tick_params(axis="x", labelsize=16, colors="dimgray", grid_linewidth=1.0)  # Proprietà Labels X Axis
    ax.tick_params(axis="y", labelsize=16, colors="dimgray", grid_linewidth=1.0)  # Proprietà Labels Y Axis

    return fig, ax  # Return oggetti fig e ax

### Grafico 4

In [10]:
def valore_correlazione_start_end(df_partenze):
    """
    Calcola il coefficiente di correlazione (Pearson), riguardante il quarto grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :returns: stringa contenente le informazioni di coefficiente e p-value
    """
    
    correlazione_str = ""  # Stringa vuota dove si concatenerà il risultato
    correlazione_pearson = sp.stats.pearsonr(df_partenze['DeltaStart'], df_partenze['DeltaEnd'])  # Calcolo il coefficiente con il metodo di Pearson
    correlazione_str += "Pearson ρ: {}".format(np.format_float_positional(correlazione_pearson[0], precision=3))  # Aggiungo alla stringa il coefficiente
    correlazione_str += "\n"  # Vado a capo
    correlazione_str += "p-value: ~{:.0E}".format(correlazione_pearson[1])  # Aggiungo alla stringa l'errore
    
    return correlazione_str  # Return stringa
    

def grafico_correlazione_start_end(df_partenze, dimensioni_grafico):
    """
    Crea il quarto grafico, della seconda parte di analisi.

    :param df_partenze: Pandas DataFame che contiene le informazioni sulle partenze in F1
    :param dimensioni_grafico: tupla contentente le dimensioni di larghezza e altezza da dare al grafico
    :returns: oggetti fig e ax (contenitore alto-livello del grafico e assi della figura)
    """
    
    fig = plt.figure(figsize=dimensioni_grafico)  # Creazione Plot Figure
    ax = plt.axes()  # Creazione Plot Axes

    vmax_count = 50
    
    sns.histplot(
        ax=ax,
        data=df_partenze,
        x='DeltaStart',
        y='DeltaEnd',
        discrete=(True, True),
        stat="count",
        vmax=vmax_count,
        linewidth=0.5,
        cmap=sns.color_palette("icefire", as_cmap=True),
        cbar=True,
        cbar_kws={"shrink": 0.75}
    )  # Plotting

    sns.regplot(
        ax=ax,
        data=df_partenze,
        x='DeltaStart',
        y='DeltaEnd',
        scatter=False,
        order=1,
        ci=100,
        color="dimgray",
        line_kws={"linewidth": 2.0, "linestyle": ":"}
    )  # Linea di regressione

    titolo = "Correlazione Partenza - Arrivo"
    ax.set_title(label=titolo, fontweight="bold", pad=20)  # Titolo Figure

    ax.set_xlabel(xlabel="Δ Start", fontstyle="italic", labelpad=15)  # Titolo X Axis
    ax.set_ylabel(ylabel="Δ End", fontstyle="italic", labelpad=15)  # Titolo Y Axis

    lim_min, lim_max = -20, +20  # Valori Globali X-Y
    ax.set_xlim(lim_min, lim_max)  # Limiti X Axis
    ax.set_ylim(lim_min, lim_max)  # Limiti Y Axis
    
    ticks = range(lim_min, lim_max + 5, 5)
    ax.set_xticks(ticks)  # Tick X Axis
    ax.set_yticks(ticks)  # Tick Y Axis

    for xt in ticks:
        ax.axvline(x=(xt - 0.5), color="white", linestyle="-", linewidth=0.5)  # Costruisco le linee griglia verticali sinistre
        ax.axvline(x=(xt + 0.5), color="white", linestyle="-", linewidth=0.5)  # Costruisco le linee griglia verticali destre
        
    for yt in ticks:
        ax.axhline(y=(yt - 0.5), color="white", linestyle="-", linewidth=0.5)  # Costruisco le linee griglia orizzontali basse
        ax.axhline(y=(yt + 0.5), color="white", linestyle="-", linewidth=0.5)  # Costruisco le linee griglia orizzontali alte
    
    ax.tick_params(axis="x", labelsize=10, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels X Axis
    ax.tick_params(axis="y", labelsize=10, colors="dimgray", grid_linewidth=0.0)  # Proprietà Labels Y Axis

    colorbar = ax.collections[0].colorbar  # Colorbar Cmap Count
    colorbar.set_label("Occorrenze", fontstyle="italic", labelpad=15)  # Titolo Colorbar
    colorbar_ticks = range(0, vmax_count + 1, 10)
    colorbar.set_ticks(colorbar_ticks)  # Ticks Colorbar
    colorbar_ticklabels = [str(t) if t != vmax_count else str(t) + "+" for t in colorbar_ticks]
    colorbar.set_ticklabels(colorbar_ticklabels, fontsize=10)  # Tick Labels Colorbar
    
    fig.tight_layout()  # Ottimizzazione Spazi Vuoti
    
    return fig, ax  # Return oggetti fig e ax